互联网中充斥着大量钓鱼欺诈类网站。这类非法网站通常试图掩人耳目、充当正规网站，而实际上却是在窃取用户的身份、密码、交易等重要信息
机器学习在信息安全领域中的一个重要应用就是用来识别这些钓鱼网站。fraudulent.csv在data文件夹中。

fraudulent.csv文件中含有10,086条数据，每条数据含有18个特征以及1个标签。各个特征的含义如下：

    contain_IP：网址中是否包涵ip，比如http://121.99.3.123/fake.html 包含ip。1表示包含，0表示不包含；
    is_long：网址字符是否过长。1表示网址过长，0表示网址不长；
    is_tinyurl：网址是否是短网址。比如https://bit.ly/2kXX6jV 就是短网址。1表示是短网址，0表示不是；
    contain_at：网址是否包含“@”符号。1表示包含，0表示不包含；
    contain_double_slash：网址是否包含“//”符号，该符号用来表示网址跳转。1表示包含，0表示不包含；
    contain_dash：网址是否包含“-”符号，该符号经常帮助用来伪装真网站，比如www.my-taobao.com 。 1表示包含，0表示不包含；
    contain_subdomain：网址是否包含子域名，比如www.ecnu.edu.cn 就包含edu和cn子域名。1表示包含，0表示不包含；
    is_SSL：网址是否是https安全链接。1表示包含，0表示不包含；
    with_long_history：网址所属的主域名存在的时间。1表示长久，0表示不长久；
    contain_icon：网址网页是否有小图标。1表示包含，0表示不包含；
    contain_ext_domain：该网页是否加载其他域名下的附件或者网页。1表示包含，0表示不包含；
    contain_email_to：该网页是否包含发送邮件的组件。1表示包含，0表示不包含；
    allow_right_click：该网页是否允许用户进行右击操作。1表示允许，0表示不允许；
    contain_pop_up_windowL：该网页是否包含弹窗。1表示包含，0表示不包含；
    contain_Iframe：该网页是否包含Iframe（嵌套网页）。1表示包含，0表示不包含；
    has_DNSRecord：网址是否有DNS记录。1表示有，0表示无；
    traffic：该网站的流量大小。1表示大，0表示小；
    google_rank：该网址在google搜索中的排名。1表示高于同类网站的平均排名，0表示低于同类网站的平均排名；
    
    y：表示网站是否是钓鱼欺诈网站，1表示是，0表示不是。
    原始数据中含有大量缺失值，请自行处理这些缺失值（可以剔除缺失值过多的列或者使用众数填充等方法）。

将原始数据分为训练集、测试集（随机种子请设置为1）（若有需要可以将训练集进一步分为训练集和验证集）。

现在请建立一个二分类模型，使用训练集训练模型，再使用测试集测试模型。

评估指标为F1值

分类模型可采用：k-近邻、决策树、逻辑回归、支持向量机等。

可以与周围同学比较一下F1值的大小（越接近1越好），看看谁的数据预处理和分类模型更强。

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.metrics import f1_score, classification_report
from sklearn.model_selection import GridSearchCV
from imblearn.over_sampling import SMOTE

# 1. 数据加载和初步探索
data = pd.read_csv(r"D:\Pycharm\PycharmProject\pythonProject\venv\dase-2024-autumn\HomeWork\data\fraudulent.csv")
print("数据形状:", data.shape)
print("\n缺失值情况:")
print(data.isnull().sum())
print("\n类别分布:")
print(data['y'].value_counts())

# 2. 数据预处理
imputer = SimpleImputer(strategy='most_frequent')
X = data.drop('y', axis=1)
y = data['y']
X_imputed = pd.DataFrame(imputer.fit_transform(X), columns=X.columns)

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X_imputed)

# 3. 处理类别不平衡
smote = SMOTE(random_state=1)
X_balanced, y_balanced = smote.fit_resample(X_scaled, y)

# 4. 数据集分割
X_train, X_test, y_train, y_test = train_test_split(
    X_balanced, y_balanced, test_size=0.2, random_state=1
)

# 5. 定义多个模型
models = {
    'RandomForest': RandomForestClassifier(random_state=1),
    'GradientBoosting': GradientBoostingClassifier(random_state=1)
}

# 6. 模型参数网格
param_grids = {
    'RandomForest': {
        'n_estimators': [100, 200],
        'max_depth': [10, 20],
        'min_samples_split': [2, 5],
        'class_weight': ['balanced']
    },
    'GradientBoosting': {
        'n_estimators': [100, 200],
        'max_depth': [3, 5],
        'learning_rate': [0.01, 0.1]
    }
}

# 7. 模型训练和评估
best_models = {}
best_scores = {}

for name, model in models.items():
    print(f"\n训练 {name}...")
    
    # 网格搜索
    grid_search = GridSearchCV(
        estimator=model,
        param_grid=param_grids[name],
        scoring='f1',
        cv=5,
        n_jobs=-1
    )
    
    # 训练模型
    grid_search.fit(X_train, y_train)
    
    # 保存最佳模型
    best_models[name] = grid_search.best_estimator_
    
    # 预测和评估
    y_pred = grid_search.predict(X_test)
    f1 = f1_score(y_test, y_pred)
    best_scores[name] = f1
    
    print(f"{name} 最佳参数:", grid_search.best_params_)
    print(f"{name} F1 分数:", f1)
    print("\n分类报告:")
    print(classification_report(y_test, y_pred))

# 8. 选择最佳模型
best_model_name = max(best_scores, key=best_scores.get)
print(f"\n最佳模型是 {best_model_name}，F1值为 {best_scores[best_model_name]}")

# 9. 特征重要性分析（针对随机森林模型）
if 'RandomForest' in best_models:
    rf_model = best_models['RandomForest']
    feature_importance = pd.DataFrame({
        'feature': X.columns,
        'importance': rf_model.feature_importances_
    })
    print("\n特征重要性:")
    print(feature_importance.sort_values('importance', ascending=False))

# 10. 保存最佳模型的预测结果
best_model = best_models[best_model_name]
final_predictions = best_model.predict(X_test)

数据形状: (10086, 19)

缺失值情况:
contain_IP                 90
is_long                    89
is_tinyurl                 88
contain_at                 82
contain_double_slash      116
contain_dash               94
contain_subdomain          97
is_SSL                     96
with_long_history        2795
contain_icon             1358
contain_ext_domain       1527
contain_email_to         2079
allow_right_click        3407
contain_pop_up_window     279
contain_Iframe            659
has_DNSRecord            1201
traffic                  1507
google_rank               664
y                           0
dtype: int64

类别分布:
y
0    6906
1    3180
Name: count, dtype: int64

训练 RandomForest...
RandomForest 最佳参数: {'class_weight': 'balanced', 'max_depth': 20, 'min_samples_split': 2, 'n_estimators': 200}
RandomForest F1 分数: 0.9208103130755064

分类报告:
              precision    recall  f1-score   support

           0       0.92      0.93      0.92      1393
           1       0.93      0.91      0.92      13